In [1]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
output_dir_4_5_5 ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.5.5_linear_scale-up_etc/ciff_sam/2021_11_23_17_59_09/count_data/'
output_dir_4_5_6 ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.5.6_no_x_factor/ciff_sam/2021_11_29_14_01_14/count_data/'
output_dir_4_5_7 ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.5.7_adjusted_x_factor_paf/ciff_sam/2021_12_06_17_17_21/count_data/'

In [3]:
def compare(dataframe):
    data_5 = pd.read_csv(output_dir_4_5_5 + dataframe + '.csv')
    data_7 = pd.read_csv(output_dir_4_5_7 + dataframe + '.csv')
    cols = [c for c in data_5.columns if c != 'value']
    return data_5.set_index(cols)-data_7.set_index(cols)

In [4]:
compare('wasting_transition_count').describe()

,value
count,241920.000000
mean,3.203563
std,25.433599
min,-309.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,585.000000


In [5]:
compare('wasting_state_person_time').describe()

,value
count,138240.000000
mean,-0.011187
std,4.084948
min,-83.542779
25%,0.000000
50%,0.000000
75%,0.000000
max,86.008214


In [6]:
def calculate_wasting_transition_rates(output_dir,
                                      ages, transition, source_state, 
                                      groupby_cols, time_to_response=False):
    wasting_pt = pd.read_csv(output_dir +'wasting_state_person_time.csv').drop(columns='Unnamed: 0')
    wasting_transitions = pd.read_csv(output_dir +'wasting_transition_count.csv').drop(columns='Unnamed: 0')
    transition = (wasting_transitions.loc[(wasting_transitions.age.isin(ages))
                                         & (wasting_transitions.measure==transition)]
                  .groupby(groupby_cols).sum())
    pt = (wasting_pt.loc[(wasting_pt.age.isin(ages))
                       & (wasting_pt.cause==source_state)]
          .groupby(groupby_cols).sum())
    transition_rate = transition / pt
    if time_to_response==True:
        transition_rate = 365 / transition_rate
    transition_rate = (transition_rate.groupby([c for c in groupby_cols if c != 'input_draw'])
                       .describe(percentiles=[0.025,0.975]))['value']
    return transition_rate

In [7]:
ages = ['6-11_months', '12_to_23_months', '2_to_4']

In [8]:
# i1 (MAM to SAM) in 4.5.5 with x-factor
calculate_wasting_transition_rates(output_dir_4_5_5, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario'], time_to_response=False)

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,1.881085,0.166318,1.611623,1.646078,1.905576,2.143461,2.190107
sqlns,12.0,1.893940,0.167979,1.622029,1.656783,1.915445,2.162153,2.210424
wasting_treatment,12.0,1.886841,0.166193,1.616425,1.651128,1.908292,2.150547,2.197140


In [9]:
#i1 (MAM to SAM) in 4.5.6 without x-factor
calculate_wasting_transition_rates(output_dir_4_5_6, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario'], time_to_response=False)

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,1.478817,0.131275,1.265059,1.292181,1.499533,1.691137,1.727720
sqlns,12.0,1.484804,0.131391,1.271335,1.298164,1.505024,1.699267,1.735679
wasting_treatment,12.0,1.481728,0.131269,1.267405,1.294545,1.504089,1.694371,1.730749


In [10]:
#i1 (MAM to SAM) in 4.5.7 without x-factor
calculate_wasting_transition_rates(output_dir_4_5_7, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario'], time_to_response=False)

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,1.468194,0.132123,1.254069,1.280498,1.488128,1.681450,1.723057
sqlns,12.0,1.477621,0.133060,1.261788,1.288401,1.497348,1.692900,1.735153
wasting_treatment,12.0,1.472479,0.131761,1.255885,1.282552,1.491118,1.684857,1.725913


In [11]:
# i1 (MAM to SAM) in 4.5.5 with x-factor
calculate_wasting_transition_rates(output_dir_4_5_5, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','x_factor'], time_to_response=False)

count      mean       std       min      2.5%  \
scenario          x_factor                                                  
baseline          cat1       12.0  2.263399  0.200477  1.939847  1.980745   
                  cat2       12.0  0.696571  0.060926  0.600500  0.610332   
sqlns             cat1       12.0  2.273005  0.202231  1.946346  1.987781   
                  cat2       12.0  0.697375  0.061679  0.598481  0.608116   
wasting_treatment cat1       12.0  2.268927  0.200079  1.944074  1.985067   
                  cat2       12.0  0.697917  0.061723  0.600144  0.610025   

                                 50%     97.5%       max  
scenario          x_factor                                
baseline          cat1      2.290835  2.581116  2.642229  
                  cat2      0.704319  0.797671  0.809668  
sqlns             cat1      2.296712  2.595219  2.657701  
                  cat2      0.705079  0.798211  0.809414  
wasting_treatment cat1      2.292664  2.586029  2.646818  
                  cat2      0.707057  0.799459  0.811886

In [12]:
# i1 (MAM to SAM) in 4.5.7 with x-factor
calculate_wasting_transition_rates(output_dir_4_5_7, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','x_factor'], time_to_response=False)

count      mean       std       min      2.5%  \
scenario          x_factor                                                  
baseline          cat1       12.0  1.760199  0.159051  1.504616  1.535217   
                  cat2       12.0  0.537613  0.046984  0.459778  0.469653   
sqlns             cat1       12.0  1.766457  0.159578  1.509496  1.539753   
                  cat2       12.0  0.538587  0.047724  0.458253  0.468571   
wasting_treatment cat1       12.0  1.763782  0.158288  1.505468  1.536173   
                  cat2       12.0  0.539001  0.047355  0.458939  0.469745   

                                 50%     97.5%       max  
scenario          x_factor                                
baseline          cat1      1.781894  2.019697  2.071600  
                  cat2      0.544072  0.616048  0.624278  
sqlns             cat1      1.788285  2.024729  2.076518  
                  cat2      0.540341  0.619481  0.628371  
wasting_treatment cat1      1.784058  2.020115  2.070455  
                  cat2      0.544643  0.618622  0.627908